In [1]:
import random
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from tqdm import tqdm

from pathlib import Path
import torchaudio

In [2]:
!git clone https://github.com/karolpiczak/ESC-50.git

Cloning into 'ESC-50'...
remote: Enumerating objects: 4199, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 4199 (delta 40), reused 38 (delta 18), pack-reused 4136
Receiving objects: 100% (4199/4199), 878.79 MiB | 32.12 MiB/s, done.
Resolving deltas: 100% (287/287), done.
Updating files: 100% (2011/2011), done.


In [5]:
path_audio = 'ESC-50/audio'

In [16]:
from torchaudio.transforms import Resample
class DataGenerator(Dataset):
    def __init__(self, path, transform = None, kind='train'):

        if kind=='train':
            files = Path(path).glob('[1-4]-*')
            self.items = [(str(file), file.name.split('-')[-1].replace('.wav', '')) for file in files]
        if kind=='test':
            files = Path(path).glob('5-*')
            self.items = [(str(file), file.name.split('-')[-1].replace('.wav', '')) for file in files]

        self.length = len(self.items)
        # print(self.length)

    def __getitem__(self, index):
        filename, label = self.items[index]
        data_tensor, orig_rate = torchaudio.load(filename)
        if orig_rate != 16000:
          resampler = Resample(orig_freq=orig_rate, new_freq=16000)
          data_tensor = resampler(data_tensor)
        tmp = data_tensor[0,0:80000]
        return (tmp, int(label))

    def __len__(self):
        return self.length

In [17]:
batch_size = 64

train_data = DataGenerator(path_audio, kind='train')
test_data = DataGenerator(path_audio, kind='test')

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [18]:
class Net(nn.Module):

    def __init__(self, num_classes=50):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(1,8), stride=(1,1), padding="same")
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(1,8), stride=(1,1), padding="same")
        self.bn2 = nn.BatchNorm2d(16)

        self.pool_1 = nn.MaxPool2d(kernel_size=(1,128), stride = (1,128), padding=0)

        self.conv3 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3,3), stride=(1,1), padding=1)
        self.bn3 = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3,3), stride=(1,1), padding=1)
        self.bn4 = nn.BatchNorm2d(32)

        self.pool_2 = nn.MaxPool2d(kernel_size=4, padding=0)

        self.conv5 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3), stride=(2,2), padding=2)
        self.bn5 = nn.BatchNorm2d(64)
        self.conv6 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3), stride=(2,2), padding=1)
        self.bn6 = nn.BatchNorm2d(64)

        self.pool_3 = nn.MaxPool2d(kernel_size=2, padding=0)

        self.conv7 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), stride=(2,2), padding=1)
        self.bn7 = nn.BatchNorm2d(128)
        self.conv8 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3,3), stride=(2,2), padding=1)
        self.bn8 = nn.BatchNorm2d(128)

        self.pool_4 = nn.MaxPool2d(kernel_size=(1,2), padding=0)
        self.dense = nn.Linear(in_features= 256, out_features=num_classes)
        self.dropout = nn.Dropout(0.2)


    def forward(self, x):
        x = x.unsqueeze(1).view(-1, 1, 1, 80000)
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool_1(x)
        x = x.view((-1,1,16, 625))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout(x)
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool_2(x)
        x = F.relu(self.bn5(self.conv5(x)))
        x = self.dropout(x)
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.pool_3(x)
        x = F.relu(self.bn7(self.conv7(x)))
        x = self.dropout(x)
        x = F.relu(self.bn8(self.conv8(x)))
        x = self.pool_4(x)
        x = x.view(x.size(0),-1)
        x = self.dense(x)
        x = self.dropout(x)
        return x

device = "cpu"
if (torch.cuda.is_available()):
    device = "cuda"

In [9]:
# summary(model,(1,80000))

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1008.)
  return F.conv2d(input, weight, bias, self.stride,


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 1, 80000]             144
       BatchNorm2d-2         [-1, 16, 1, 80000]              32
           Dropout-3         [-1, 16, 1, 80000]               0
            Conv2d-4         [-1, 16, 1, 80000]           2,064
       BatchNorm2d-5         [-1, 16, 1, 80000]              32
         MaxPool2d-6           [-1, 16, 1, 625]               0
            Conv2d-7          [-1, 32, 16, 625]             320
       BatchNorm2d-8          [-1, 32, 16, 625]              64
           Dropout-9          [-1, 32, 16, 625]               0
           Conv2d-10          [-1, 32, 16, 625]           9,248
      BatchNorm2d-11          [-1, 32, 16, 625]              64
        MaxPool2d-12           [-1, 32, 4, 156]               0
           Conv2d-13            [-1, 64, 3, 79]          18,496
      BatchNorm2d-14            [-1, 64

In [19]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0
    print("------------------------------- Epoch:", epoch,"-------------------------------")
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()

        output = model(data.to(device))

        loss = loss_criteria(output, target)

        train_loss += loss.item()

        loss.backward(retain_graph=True)
        optimizer.step()

    avg_loss = train_loss / (batch_idx+1)
    print('Training set: Average loss: {:.6f}'.format(avg_loss))
    return avg_loss

In [20]:
loss_criteria = nn.CrossEntropyLoss()
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        batch_count = 0
        for data, target in test_loader:
            batch_count += 1
            data, target = data.to(device), target.to(device)

            output = model(data)

            test_loss += loss_criteria(output, target).item()

            _, predicted = torch.max(output.data, 1)
            correct += torch.sum(target==predicted).item()

    avg_loss = test_loss / batch_count
    print('Validation set: Average loss: {:.6f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        avg_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    return avg_loss

In [21]:
model = Net(num_classes=50).to(device)
input_data = torch.randn(64, 1, 80000)

optimizer = optim.Adam(model.parameters(), lr=3e-4)

loss_criteria = nn.CrossEntropyLoss()

epoch_nums = []
training_loss = []
validation_loss = []

epochs = 50
print('Training on', device)
for epoch in tqdm(range(1, epochs + 1)):
    train_loss = train(model, device, train_loader, optimizer, epoch)
    test_loss = test(model, device, test_loader)
    epoch_nums.append(epoch)
    training_loss.append(train_loss)
    validation_loss.append(test_loss)

Training on cuda


  0%|          | 0/50 [00:00<?, ?it/s]

------------------------------- Epoch: 1 -------------------------------
Training set: Average loss: 3.943439


  2%|▏         | 1/50 [00:34<27:55, 34.20s/it]

Validation set: Average loss: 3.922614, Accuracy: 8/400 (2%)

------------------------------- Epoch: 2 -------------------------------
Training set: Average loss: 3.732102


...

Validation set: Average loss: 2.051270, Accuracy: 171/400 (43%)

------------------------------- Epoch: 49 -------------------------------
Training set: Average loss: 1.234426


 98%|█████████▊| 49/50 [28:27<00:34, 34.88s/it]

Validation set: Average loss: 2.109373, Accuracy: 165/400 (41%)

------------------------------- Epoch: 50 -------------------------------
Training set: Average loss: 1.264724


100%|██████████| 50/50 [29:01<00:00, 34.83s/it]

Validation set: Average loss: 2.020796, Accuracy: 175/400 (44%)



In [23]:
from sklearn.metrics import precision_score, recall_score, f1_score
import torch
import torch.nn as nn

loss_criteria = nn.CrossEntropyLoss()

def test_upd(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    all_targets = []
    all_predictions = []

    with torch.no_grad():
        batch_count = 0
        for data, target in test_loader:
            batch_count += 1
            data, target = data.to(device), target.to(device)

            output = model(data)
            test_loss += loss_criteria(output, target).item()

            _, predicted = torch.max(output.data, 1)
            correct += torch.sum(target == predicted).item()

            all_targets.extend(target.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    avg_loss = test_loss / batch_count
    accuracy = 100. * correct / len(test_loader.dataset)

    # Calculate precision, recall and F1-score
    precision = precision_score(all_targets, all_predictions, average='weighted')
    recall = recall_score(all_targets, all_predictions, average='weighted')
    f1 = f1_score(all_targets, all_predictions, average='weighted')

    print('Validation set: Average loss: {:.6f}, Accuracy: {}/{} ({:.0f}%), Precision: {:.4f}, Recall: {:.4f}, F1 Score: {:.4f}\n'.format(
        avg_loss, correct, len(test_loader.dataset), accuracy, precision, recall, f1))

    return avg_loss, accuracy, precision, recall, f1

In [24]:
t = test_upd(model, device, test_loader)

Validation set: Average loss: 2.105532, Accuracy: 175/400 (44%), Precision: 0.4823, Recall: 0.4375, F1 Score: 0.4316

